In [1]:
#Script for scraping data from IMDB , Youtube and Box Offfice 
#Final analysis is in the blog post below
#https://medium.com/@chawannut157/will-avenger-infinity-war-surpass-titanic-633638c9ac24
#Might need to adjust IP in youtube API

import requests as rq
from bs4 import BeautifulSoup
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
#Construct URL for scraping top 700 box office movies
all_url = []
temp = 'http://www.boxofficemojo.com/alltime/world/?pagenum'
#temp
for i in range(8):
    all_url.append(temp + '=' + str(i+1) + '&p=.htm')
    #print(temp + '=' + str(i+1) + '&p=.htm' )
all_url

#For test use only the first two URLs
#Delete this line if scrape everything
all_url = all_url[0:2]

In [3]:
#This chunk if for scraping name and revenue
name = []
rev = []

for k in all_url:
    r = rq.get(k)
    html = r.text
    soup = BeautifulSoup(html, "html5lib")
    type(soup)
    a = soup.findAll('b')
    a = a[3:len(a)-3]
    for i in range(len(a)):
        if i%2 == 0:
            name.append(a[i].get_text())
        else:
            rev.append(float(a[i].get_text()[1:].replace(",","")))
    
name = np.array(name).reshape(-1,1)
rev = np.array(rev).reshape(-1,1)

In [4]:
df = pd.DataFrame({'name' :name.ravel()  ,
                              'rev': rev.ravel().astype('float')})
df.head()

,name,rev
0,Avatar,2788.0
1,Titanic,2187.5
2,Star Wars: The Force Awakens,2068.2
3,Jurassic World,1671.7
4,Marvel's The Avengers,1518.8


In [5]:
#Getting Distributor , Genre , Ratings and Budget

distributor = []
genre = []
rating  = []
budget = []

for url in all_url:
    r = rq.get(url)
    html = r.text
    soup = BeautifulSoup(html, "html5lib")
    #type(soup)
    a = soup.findAll('a' , href = True)
    link_text = [a['href'] for a in soup.find_all('a', href=True) if a.text.strip()]
    
    for m in range(38,len(link_text)-27):
        #print(link_text[m])
        new_url =  "http://www.boxofficemojo.com" + link_text[m]
        r = rq.get(new_url)
        html = r.text
        soup = BeautifulSoup(html, "html5lib")
        a = soup.find("table" , {"style" : "padding-top: 5px;"}  )
        children = a.findAll("td" , {"valign" : "top"})
        
        distributor.append(children[2].get_text())
        genre.append(children[4].get_text())
        rating.append(children[6].get_text())
        budget.append(children[7].get_text())

print(distributor[0:5])
print(genre[0:5])
print(rating[0:5])
print(budget[0:5])


['Distributor: Fox', 'Distributor: Paramount', 'Distributor: Buena Vista', 'Distributor: Universal', 'Distributor: Buena Vista']
['Genre: Sci-Fi Adventure', 'Genre: Romance', 'Genre: Sci-Fi Fantasy', 'Genre: Action / Adventure', 'Genre: Action / Adventure']
['MPAA Rating: PG-13', 'MPAA Rating: PG-13', 'MPAA Rating: PG-13', 'MPAA Rating: PG-13', 'MPAA Rating: PG-13']
['Production Budget: $237 million', 'Production Budget: $200 million', 'Production Budget: $245 million', 'Production Budget: $150 million', 'Production Budget: $220 million']


In [6]:
#Clean up text 

genre_new , Distributor_new , rating_new , budget_new = [] , [] , [] , []
for i in range(len(genre)):
    genre_new.append(genre[i][genre[i].find(":")+1:])
    Distributor_new.append(distributor[i][distributor[i].find(":")+1:])
    rating_new.append(rating[i][rating[i].find(":")+1:])
    budget_new.append(budget[i][budget[i].find("$")+1:].split()[0])

genre_new = np.array(genre_new)
Distributor_new = np.array(Distributor_new)
rating_new = np.array(rating_new)
budget_new = np.array(budget_new)
budget_new[np.where(budget_new== 'Production')] = "NaN"


In [7]:
#dat = pd.DataFrame(genre_new ,  Distributor_new , rating_new  )
df['genre'] =  genre_new.ravel() 
df['Distributor'] =  Distributor_new.ravel()
df['Rating'] =  rating_new.ravel()
df['Budget'] = budget_new.ravel()
df['Budget'].replace('NaN' , np.NaN)
df['Budget'] = pd.to_numeric(df['Budget'] , errors='coerce')
#budget_new.replace('NaN', np.NaN)
df.head()

,name,rev,genre,Distributor,Rating,Budget
0,Avatar,2788.0,Sci-Fi Adventure,Fox,PG-13,237.0
1,Titanic,2187.5,Romance,Paramount,PG-13,200.0
2,Star Wars: The Force Awakens,2068.2,Sci-Fi Fantasy,Buena Vista,PG-13,245.0
3,Jurassic World,1671.7,Action / Adventure,Universal,PG-13,150.0
4,Marvel's The Avengers,1518.8,Action / Adventure,Buena Vista,PG-13,220.0


In [8]:
#get year 
year = []
for k in all_url:
    r = rq.get(k)
    html = r.text
    soup = BeautifulSoup(html, "html5lib")
    a = soup.findAll('td' , {"align" : "center" } , {"size" : "2"})
    for i in range(9 , len(a),  2):
        temp = (a[i].get_text())[0:4]
        year.append(temp)

In [9]:
df['Year'] = year
df.head()


,name,rev,genre,Distributor,Rating,Budget,Year
0,Avatar,2788.0,Sci-Fi Adventure,Fox,PG-13,237.0,2009
1,Titanic,2187.5,Romance,Paramount,PG-13,200.0,1997
2,Star Wars: The Force Awakens,2068.2,Sci-Fi Fantasy,Buena Vista,PG-13,245.0,2015
3,Jurassic World,1671.7,Action / Adventure,Universal,PG-13,150.0,2015
4,Marvel's The Avengers,1518.8,Action / Adventure,Buena Vista,PG-13,220.0,2012


In [ ]:
#Test
import sys
sys.path

In [13]:
#Get youtube data 
# Scraping Youtube View 
#sys.path.remove("C:\\Users\\USA\\youtube_tutorial")
from apiclient.discovery import build #pip install google-api-python-client
from apiclient.errors import HttpError #pip install google-api-python-client
from oauth2client.tools import argparser #pip install oauth2client
sys.path.append("C:\\Users\\USA\\youtube_tutorial")
from youtube_videos import youtube_search
from youtube_videos import geo_query
import json


In [15]:
#Need to change API_KEY IN youtube_tutorial + also need to add current IP address on Youube API Console
viewCount = likeCount = dislikeCount = commentCount = 0 
viewCount_store  = []
likeCount_store = [] 
dislikeCount_store = [] 
commentCount_store = []

for idx , movie in enumerate(name):
    print(idx , movie)
    test = youtube_search(str(movie)[2:-2] + " Trailers")
    #Extract only top three result
    for video in test[1][0:3]:
        #print (video['id']['videoId'])
        k = geo_query(video['id']['videoId'])
       # print(k['items'][0]['statistics'])
        #print(viewCount)
        try:
            viewCount +=   int(k['items'][0]['statistics']['viewCount'])
            likeCount +=  int(k['items'][0]['statistics']['likeCount'])
            dislikeCount += int(k['items'][0]['statistics']['dislikeCount'])
            commentCount += int(k['items'][0]['statistics']['commentCount'])
        except Exception:
            likeCount +=  0
            dislikeCount += 0
            commentCount += 0
        
    #print(viewCount)
    viewCount_store.append(viewCount)
    #print(viewCount_store)
    likeCount_store.append(likeCount)
    dislikeCount_store.append(dislikeCount)
    commentCount_store.append(commentCount)
    viewCount = 0 
    likeCount = 0 
    dislikeCount = 0 
    commentCount = 0
        #dislikeCount += dislikeCount + int(k['items'][0]['statistics']['dislikeCount'])
        
        #print(commentCount)
        #print(likeCount)/        #print(k['items'][0]['statistics'])

#test


0 ['Avatar']
1 ['Titanic']
2 ['Star Wars: The Force Awakens']
3 ['Jurassic World']
4 ["Marvel's The Avengers"]
5 ['Furious 7']
6 ['Avengers: Age of Ultron']
7 ['Harry Potter and the Deathly Hallows Part 2']
8 ['Frozen']
9 ['Beauty and the Beast (2017)']
10 ['The Fate of the Furious']
11 ['Star Wars: The Last Jedi']
12 ['Iron Man 3']
13 ['Minions']
14 ['Captain America: Civil War']
15 ['Transformers: Dark of the Moon']
16 ['The Lord of the Rings: The Return of the King']
17 ['Skyfall']
18 ['Transformers: Age of Extinction']
19 ['The Dark Knight Rises']
20 ['Toy Story 3']
21 ["Pirates of the Caribbean: Dead Man's Chest"]
22 ['Rogue One: A Star Wars Story']
23 ['Pirates of the Caribbean: On Stranger Tides']
24 ['Despicable Me 3']
25 ['Jurassic Park']
26 ['Finding Dory']
27 ['Star Wars: Episode I - The Phantom Menace']
28 ['Alice in Wonderland (2010)']
29 ['Zootopia']
30 ['The Hobbit: An Unexpected Journey']
31 ['The Dark Knight']
32 ["Harry Potter and the Sorcerer's Stone"]
33 ['Despicabl

In [16]:
df['view'] , df['like'] , df['dislike'] , df['comment']= viewCount_store , likeCount_store , dislikeCount_store , commentCount_store
df['Year'] = pd.to_numeric(df['Year'])
df.dtypes

name            object
rev            float64
genre           object
Distributor     object
Rating          object
Budget         float64
Year             int64
view             int64
like             int64
dislike          int64
comment          int64
dtype: object

In [18]:
df.to_csv('all_data_movies_github.csv')
df.head()

,name,rev,genre,Distributor,Rating,Budget,Year,view,like,dislike,comment
0,Avatar,2788.0,Sci-Fi Adventure,Fox,PG-13,237.0,2009,19969689,22516,2438,10470
1,Titanic,2187.5,Romance,Paramount,PG-13,200.0,1997,16946495,40140,1828,10625
2,Star Wars: The Force Awakens,2068.2,Sci-Fi Fantasy,Buena Vista,PG-13,245.0,2015,185296041,1098560,44362,209189
3,Jurassic World,1671.7,Action / Adventure,Universal,PG-13,150.0,2015,134024200,930345,47813,127443
4,Marvel's The Avengers,1518.8,Action / Adventure,Buena Vista,PG-13,220.0,2012,146332842,2919978,62906,401853


In [19]:
from imdb import IMDb

In [20]:
import time
rating_avg = []
m_temp = []
ia = IMDb()
for mov in df['name']:    
    try:
            movies = ia.search_movie(mov)
    except Exception:
            m_temp.append(mov)
            rating_avg.append(0)
    
    try:
            movies = ia.search_movie(mov) 
            item = movies[0]
            ia.update(item)
            m_temp.append(mov)
            rating_avg.append(item['rating'])
    except Exception:
            pass
    #Ensure that everything match
    print("Movie at " , len(m_temp) , " Rating at ", len(rating_avg))
    #print(item['rating'])
    #time.sleep(np.random.random(1))

Movie at  1  Rating at  1
Movie at  2  Rating at  2
Movie at  3  Rating at  3
Movie at  4  Rating at  4
Movie at  5  Rating at  5
Movie at  6  Rating at  6
Movie at  7  Rating at  7
Movie at  8  Rating at  8
Movie at  9  Rating at  9
Movie at  10  Rating at  10
Movie at  11  Rating at  11
Movie at  12  Rating at  12
Movie at  13  Rating at  13
Movie at  14  Rating at  14
Movie at  15  Rating at  15
Movie at  16  Rating at  16
Movie at  17  Rating at  17
Movie at  18  Rating at  18
Movie at  19  Rating at  19
Movie at  20  Rating at  20
Movie at  21  Rating at  21
Movie at  22  Rating at  22
Movie at  23  Rating at  23
Movie at  24  Rating at  24
Movie at  25  Rating at  25
Movie at  26  Rating at  26
Movie at  27  Rating at  27
Movie at  28  Rating at  28
Movie at  29  Rating at  29
Movie at  30  Rating at  30
Movie at  31  Rating at  31
Movie at  32  Rating at  32
Movie at  33  Rating at  33
Movie at  34  Rating at  34
Movie at  35  Rating at  35
Movie at  36  Rating at  36
Movie at  

In [21]:
#Other data is available as well but not used at the moment
#item = movies[0]
#item
#ia.update(item)
#ia.update(item , 'goofs')
#print(item.keys())
#print(item['votes'])
#print(item['plot outline'])
#print(item['votes'])
#print(item['title'])
#print(item['median'])

#for item in movies:
#print(item['long imdb canonical title'], item.movieID)
#for item in movies:
# print(item['rating'])


['title', 'kind', 'year', 'genres', 'runtimes', 'countries', 'country codes', 'language codes', 'color info', 'aspect ratio', 'sound mix', 'certificates', 'rating', 'votes', 'cover url', 'plot outline', 'languages', 'akas', 'plot', 'canonical title', 'long imdb title', 'long imdb canonical title', 'smart canonical title', 'smart long imdb canonical title', 'full-size cover url']
577112
An art collector appeals to Jones to embark on a search for the Holy Grail. He learns that another archaeologist has disappeared while searching for the precious goblet, and the missing man is his own father, Dr. Henry Jones. The artifact is much harder to find than they expected, and its powers are too much for those impure in heart.
577112
Indiana Jones and the Last Crusade


In [ ]:
#rating_avg.append(8.0)
#rating_avg.append(6.7)


In [ ]:
#rating_avg
#vowel = ['a', 'e', 'i', 'u']
# inserting element to list at 4th position
#vowel.insert(3, 'o')
#rating_avg.insert(177, 8.0)
#rating_avg.insert(378, 6.7)


In [23]:
#rating_avg_2 = rating_avg[:718]
df.head()

,name,rev,genre,Distributor,Rating,Budget,Year,view,like,dislike,comment
0,Avatar,2788.0,Sci-Fi Adventure,Fox,PG-13,237.0,2009,19969689,22516,2438,10470
1,Titanic,2187.5,Romance,Paramount,PG-13,200.0,1997,16946495,40140,1828,10625
2,Star Wars: The Force Awakens,2068.2,Sci-Fi Fantasy,Buena Vista,PG-13,245.0,2015,185296041,1098560,44362,209189
3,Jurassic World,1671.7,Action / Adventure,Universal,PG-13,150.0,2015,134024200,930345,47813,127443
4,Marvel's The Avengers,1518.8,Action / Adventure,Buena Vista,PG-13,220.0,2012,146332842,2919978,62906,401853


In [ ]:
#set(df['name']) - set(m_temp)
#df[df['name'] == "The Intouchables (U.S.-only)"]
#rating_avg_2.insert(234,8.6)

In [ ]:
#df['rating'] = rating_avg_2
#df.tail(5)
df.to_csv('all_data_movies_final.csv')